Quora duplicate questions dataset

In [14]:
import pandas as pd

df1 = pd.read_csv('..\\csvs\\quora_questions.csv')

df1.shape

(404351, 6)

In [40]:
import pandas as pd

df = pd.read_csv('..\\csvs\\quora_questions.csv')

df = df.sample(n=40000,random_state=42)

df.head()

,id,qid1,qid2,question1,question2,is_duplicate
120567,120567,238932,238933,How does the Boggart work?,What would the boggart of a boggart be?,0
324466,324466,636476,636477,What is difference between project manager and...,What are the differences between project manag...,0
398558,398558,778728,778729,What hotel in Jabalpur would be safe for unmar...,What hotel in Allahabad would be safe for unma...,0
339914,339914,666314,666315,What is stronger - Super Saiyan 4 or Super Sai...,How does Gohan turn into Super Saiyan 2?,0
185732,185732,366764,366765,How do I fill in Address Line 1 and Address Li...,How do I register desired web address?,0


In [41]:
q1 = df['question1'].values.tolist()
q2 = df['question2'].values.tolist()

In [42]:
print(df.duplicated().sum())
print(df.isnull().sum())

0
id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64


In [ ]:
# remove duplicated and null rows

df.drop_duplicates(inplace=True)
df.dropna(inplace=True)

NameError: name 'df' is not defined

In [44]:
# see data distribution

val = df['is_duplicate'].value_counts()
zc=val[0]
oc=val[1]
print(f"Percentage of 0: {zc*100/(zc+oc):.2f}%")
print(f"Percentage of 1: {oc*100/(zc+oc):.2f}%")

Percentage of 0: 63.06%
Percentage of 1: 36.94%


In [45]:
X = df[['question1','question2']].values
Y = df['is_duplicate'].values

In [46]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

In [21]:
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
import warnings

warnings.filterwarnings('ignore')

stop_words = set(stopwords.words('english'))

def remove_html_tags(text):
    """Remove HTML tags using BeautifulSoup"""
    if not isinstance(text, str) or not text:
        return text
    
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text()

exclude = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

def remove_punctuation(s:str)->str:
    return s.translate(str.maketrans('','',exclude))

def remove_url(s: str) -> str:
    """Remove URLs from string - simple and effective"""
    if not isinstance(s, str) or not s:
        return s
    
    # Pattern that catches most URLs
    url_pattern = r'http[s]?://\S+|www\.\S+|\S+\.(com|org|net|edu|gov|io|co)\S*'
    
    # Remove URLs
    clean_text = re.sub(url_pattern, '', s, flags=re.IGNORECASE)
    
    # Clean up spaces
    return ' '.join(clean_text.split()) 

def pre_process_paragraph(s:str)->list[str]:
    # lowercase and stripping
    s = s.strip().lower()

    # removing urls, punctuations and html tags + [math] substrings
    s = re.sub(r'\[math\].*?\[/math\]', '', s)
    s = remove_url(remove_punctuation(remove_html_tags(s)))

    # Clean up extra whitespace
    s = ' '.join(s.split())

    # word level tokenize
    tokenized_para = word_tokenize(s)

    # removing stopwords
    tokenized_para = [ele for ele in tokenized_para if ele not in stop_words]

    # spell checker (skip for performance)

    # stemming (skip for performance)

    return tokenized_para

In [22]:
corpus = [pre_process_paragraph(q1) for q1,_ in x_train]
corpus.extend([pre_process_paragraph(q2) for _,q2 in x_train])

In [23]:
corpus.__len__()

160000

In [14]:
import gensim

model = gensim.models.Word2Vec(workers=6,window=3,vector_size=300)

In [15]:
model.build_vocab(corpus)

In [16]:
model.train(corpus,epochs=model.epochs,total_examples=model.corpus_count)

(16784397, 17956735)

In [ ]:
# save word2vec model
import joblib

joblib.dump(model,'..\\models\\w2v_duplicate_.pkl')

['..\\models\\w2v_duplicate_.pkl']

In [18]:
import fuzzywuzzy.fuzz as fuzz
import joblib

model = joblib.load('..\\models\\w2v_duplicate_.pkl')

def word_level_w2v(s):
    """Get word vector with error handling"""
    try:
        return model.wv[s]
    except KeyError:
        # Return zero vector if word not in vocabulary
        return np.zeros(model.wv.vector_size)

def getCommonWords(s1:str,s2:str):
    st1 = set(s1.split())
    st2 = set(s2.split())
    
    return len(st1 & st2)

def getCommonStopwords(s1:str,s2:str):
    words1 = set(s1.split())
    words2 = set(s2.split())

    return len(words1 & stop_words & words2)

def getTotalUniqueWords(s1:str,s2:str):
    st1 = set(s1.split())
    st2 = set(s2.split())
    
    return len(st1) + len(st2)

def getStopwordsCount(s1:str):
    return len(stop_words & set(s1.split()))

def getLongestSubstringRatio(s1: str, s2: str) -> float:
    """Optimized longest common substring ratio calculation"""
    if not s1 or not s2:
        return 0.0
    
    s1, s2 = s1.lower(), s2.lower()
    m, n = len(s1), len(s2)
    
    # Use rolling arrays to save memory
    prev = [0] * (n + 1)
    curr = [0] * (n + 1)
    
    max_length = 0
    
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if s1[i-1] == s2[j-1]:
                curr[j] = prev[j-1] + 1
                max_length = max(max_length, curr[j])
            else:
                curr[j] = 0
        
        # Swap arrays
        prev, curr = curr, prev
    
    min_length = min(m, n)
    return max_length / min_length if min_length > 0 else 0.0

def getFuzzyMetrics(s1,s2):
    # fuzz ratio
    fuzz_ratio = fuzz.ratio(s1,s2)

    # fuzz_partial_ratio
    fuzz_partial_ratio = fuzz.partial_ratio(s1,s2)

    # fuzz_sort_ratio
    fuzz_set_ratio = fuzz.token_sort_ratio(s1,s2)

    # fuzz set ratio
    fuzz_token_ratio = fuzz.token_set_ratio(s1,s2)

    return fuzz_ratio,fuzz_partial_ratio,fuzz_set_ratio,fuzz_token_ratio

def w2v_sentence(s1:str,s2:str):
    tokens_s1 = pre_process_paragraph(s1)
    tokens_s2 = pre_process_paragraph(s2)

    # 1. self engineered features

    n_chars1 = len(s1)
    n_chars2 = len(s2)
    n_words1 = len(s1.split())
    n_words2 = len(s2.split())
    common_words = getCommonWords(s1,s2)
    total_unique_words = getTotalUniqueWords(s1,s2)
    common_ratio = common_words / total_unique_words
    common_stopwords = getCommonStopwords(s1,s2)
    stopwords1 = getStopwordsCount(s1)
    stopwords2 = getStopwordsCount(s2)
    common_tokens = len(set(tokens_s1) & set(tokens_s2))
    tokens1 = len(set(tokens_s1))
    tokens2 = len(set(tokens_s2))

    # 1.1 token level features
    cwc_min = common_words/min(n_words1,n_words2)
    cwc_max = common_words/max(n_words1,n_words2)
    csc_min = common_stopwords/min(stopwords1,stopwords2) if min(stopwords1,stopwords2) else 0.0
    csc_max = common_stopwords/max(stopwords1,stopwords2) if max(stopwords1,stopwords2) else 0.0
    ctc_min = common_tokens/min(tokens1,tokens2) if min(tokens1,tokens2) else 0.0
    ctc_max = common_tokens/max(tokens1,tokens2) if max(tokens1,tokens2) else 0.0
    last_word_eq = bool(s1.split()[-1]==s2.split()[-1])
    first_word_eq = bool(s1.split()[0]==s2.split()[0])

    mean_length = (n_words1 + n_words2) / 2
    abs_len_diff = abs(n_words2 - n_words1)
    longest_substring_ratio = getLongestSubstringRatio(s1,s2)

    fuzz_ratio,fuzz_partial_ratio,fuzz_set_ratio,fuzz_token_ratio = getFuzzyMetrics(s1,s2)

    v1 = [word_level_w2v(ele) for ele in tokens_s1] or [np.zeros(model.wv.vector_size)]
    v2 = [word_level_w2v(ele) for ele in tokens_s2] or [np.zeros(model.wv.vector_size)]

    # additional features vector
    additional_vec = [n_chars1,n_chars2,n_words1,n_words2,common_words,total_unique_words,common_ratio]
    additional_vec.extend([cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq])
    additional_vec.extend([mean_length,abs_len_diff,longest_substring_ratio])
    additional_vec.extend([fuzz_ratio,fuzz_partial_ratio,fuzz_set_ratio,fuzz_token_ratio])

    final_feature_vector = np.mean(np.concatenate([v1,v2]),axis=0)
    final_feature_vector = np.concatenate([final_feature_vector,additional_vec])

    return final_feature_vector

In [61]:
getCommonStopwords('this is akash','this here is akash and papa')

2

In [19]:
w2v_sentence('i want to become ias','how do i become an ias')[-10:]

array([ 0.66666667,  1.        ,  0.        ,  5.5       ,  1.        ,
        0.4       , 57.        , 69.        , 57.        , 76.        ])

In [63]:
x_train_real = [w2v_sentence(s1,s2) for s1,s2 in x_train]
x_test_real = [w2v_sentence(s1,s2) for s1,s2 in x_test]

In [65]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=20,              
    min_samples_split=10,       
    min_samples_leaf=5,           
    random_state=42,                 
    class_weight='balanced'
)

rf.fit(x_train_real,y_train)

,n_estimators,300
,criterion,'gini'
,max_depth,20
,min_samples_split,10
,min_samples_leaf,5
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [95]:
# predictions
import joblib
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

rf_model = joblib.load('..\\models\\duplicate_.pkl')

y_pred = rf_model.predict(x_test_real)

print(f"Accuracy: {accuracy_score(y_pred=y_pred,y_true=y_test):.4f}")
print(f"Precision: {precision_score(y_pred=y_pred,y_true=y_test):.4f}")
print(f"Recall: {recall_score(y_pred=y_pred,y_true=y_test):.4f}")
print(f"F1 score: {f1_score(y_pred=y_pred,y_true=y_test):.4f}")

Accuracy: 0.8089
Precision: 0.6868
Recall: 0.8793
F1 score: 0.7713


In [67]:
# save model
import joblib

joblib.dump(rf,'..\\models\\duplicate_.pkl')

['..\\models\\duplicate_.pkl']

In [20]:
import joblib
from time import time

start = time()

rf_model = joblib.load('..\\models\\duplicate_.pkl')

s1 = 'where do i learn data science'
s2 = 'resources to get started with data science'

sample1 = w2v_sentence(s1,s2)

y_pred = rf_model.predict([sample1])
print("Duplicate" if y_pred==1 else 'Non Duplicate')
print(f"Prediction took {time()-start:.4f} seconds")

Duplicate
Prediction took 0.7340 seconds


Model using PyTorch

In [24]:
print(f"Number of unique elements in corpus: {len(set(tuple(x) for x in corpus))}")

Number of unique elements in corpus: 131152


In [25]:
vocab = {'<UNK>':0}
for tokenized_sentence in corpus:
    for word in tokenized_sentence:
        if word not in vocab:
            vocab[word] = len(vocab)

In [26]:
def para_to_indices(s:str):
    arr = []
    for word in pre_process_paragraph(s):
        arr.append(0 if word not in vocab else vocab[word])
    return arr

In [27]:
import numpy as np

x_train = [np.concatenate([para_to_indices(sen1),para_to_indices(sen2)]) for sen1,sen2 in x_train]
x_test = [np.concatenate([para_to_indices(sen1),para_to_indices(sen2)]) for sen1,sen2 in x_test]

In [28]:
from torch.utils.data import Dataset,DataLoader
import torch

class CustomDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        # self.x[index] is already a concatenated numpy array of indices
        return torch.tensor(self.x[index], dtype=torch.long), torch.tensor(self.y[index], dtype=torch.float32)

In [29]:
train_dataset = CustomDataset(x_train,y_train)
test_dataset = CustomDataset(x_test,y_test)

In [30]:
import torch

torch.manual_seed(42)

In [32]:
import torch
from torch.nn.utils.rnn import pad_sequence

def collate_pad(batch):
    """
    Collate function to pad sequences in a batch to the same length.
    Args:
        batch: list of (tensor, label) tuples
    Returns:
        padded_tensors: tensor of shape (batch_size, max_seq_len)
        labels: tensor of shape (batch_size,)
    """
    sequences, labels = zip(*batch)
    # Pad sequences to the length of the longest sequence in the batch
    padded_tensors = pad_sequence(sequences, batch_first=True, padding_value=0)
    labels = torch.stack(labels)
    return padded_tensors, labels

train_loader = DataLoader(train_dataset,batch_size=32,collate_fn=collate_pad)
test_loader = DataLoader(test_dataset,batch_size=32,shuffle=True,collate_fn=collate_pad)

In [33]:
from torch import nn

In [34]:
class MyNN(nn.Module):
    def __init__(self, embedding_dim=150, dropout=0.3, hidden_size=128):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=len(vocab), embedding_dim=embedding_dim)
        self.lstm = nn.LSTM(
            embedding_dim,
            num_layers=4,
            dropout=dropout,
            hidden_size=hidden_size,
            batch_first=True,
            bidirectional=True
        )
        self.dropout = nn.Dropout(p=dropout)
        self.linear = nn.Linear(hidden_size * 2, 1)  # hidden_size * 2 for bidirectional

    def forward(self, x):
        embedding = self.embedding(x)
        _, (hd, _) = self.lstm(embedding)
        # hd shape: (num_layers * num_directions, batch, hidden_size)
        # For bidirectional, concatenate last layer's forward and backward hidden states
        out = torch.cat((hd[-2], hd[-1]), dim=1)  # shape: (batch, hidden_size*2)
        out = self.dropout(out)
        return self.linear(out).squeeze(-1)

In [93]:
from torch import nn

class MyNN(nn.Module):
    def __init__(self, embedding_dim=256, num_heads=4, num_layers=4, dropout=0.3):
        super().__init__()
        self.embedding = nn.Embedding(len(vocab), embedding_dim)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embedding_dim,
            nhead=num_heads,
            dropout=dropout,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.linear = nn.Linear(embedding_dim, 1)

    def forward(self, x):
        emb = self.embedding(x)
        out = self.transformer(emb)
        out = out.mean(dim=1)  # Pooling
        return self.linear(out).squeeze(-1)


In [55]:
embedding = nn.Embedding(len(vocab), 100)
encoder_layer = nn.TransformerEncoderLayer(
    d_model=100,
    nhead=4,
    dropout=0.3,
    batch_first=True
)
transformer = nn.TransformerEncoder(encoder_layer, num_layers=2)
linear = nn.Linear(100, 1)

In [37]:
eg = train_dataset[0][0]
eg

tensor([    1,     2,     3,     4,     1,     2,     3, 36321])

In [38]:
eg = 0
for ele in train_loader:
    eg = ele[0]
    break

In [39]:
out = embedding(eg)
out.shape

torch.Size([32, 23, 100])

In [40]:
out1 = transformer(out)
out1.shape

torch.Size([32, 23, 100])

In [162]:
out = dropout(hn[-1])
out.shape

torch.Size([32, 256])

In [42]:
out2 = linear(out1)
out2.shape

torch.Size([32, 23, 1])

In [164]:
out.shape

torch.Size([32, 1])

In [94]:
neta = 2e-4
lambda_ = 3e-4
epochs = 5

In [95]:
# model and device
model = MyNN()
device = torch.device('cuda')
model.to(device)

# optimizer
optimizer = torch.optim.Adam(model.parameters(),lr=neta,weight_decay=lambda_)

# loss fn
loss_fn = nn.BCEWithLogitsLoss()

In [96]:
# training loop

for epoch in range(epochs):
    total_loss = 0.0
    for train_data,train_label in train_loader:
        # move to gpu
        train_data,train_label = train_data.to(device),train_label.to(device)

        # clear gradients
        optimizer.zero_grad()

        # forward pass
        out = model(train_data)

        # loss calculation
        loss = loss_fn(out,train_label)

        # backward pass
        loss.backward()

        # update gradients
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch: {epoch+1} ====> Loss: {total_loss/len(train_label):.4f}")
        

Epoch: 1 ====> Loss: 46.1961
Epoch: 2 ====> Loss: 43.3003
Epoch: 3 ====> Loss: 41.9129
Epoch: 4 ====> Loss: 40.7463
Epoch: 5 ====> Loss: 39.2650


In [97]:
# Fixed evaluation function
def evaluate_rnn_model():
    model.eval()
    total = 0
    correct = 0

    with torch.no_grad():
        for test_batch, test_labels in test_loader:
            test_batch, test_labels = test_batch.to(device), test_labels.to(device)

            y_pred = model(test_batch)
            
            # ✅ Fixed prediction calculation for sigmoid output
            predictions = (torch.sigmoid(y_pred) > 0.5).float()
            
            correct += (predictions == test_labels).sum().item()
            total += test_labels.size(0)

    accuracy = 100 * correct / total
    print(f"🎯 Test Accuracy: {accuracy:.2f}%")
    return accuracy

# Run evaluation
print("📊 Evaluating model on training data...")
test_accuracy = evaluate_rnn_model()

📊 Evaluating model on training data...
🎯 Test Accuracy: 73.77%


In [98]:
# Fixed evaluation function
def evaluate_rnn_model():
    model.eval()
    total = 0
    correct = 0

    with torch.no_grad():
        for test_batch, test_labels in train_loader:
            test_batch, test_labels = test_batch.to(device), test_labels.to(device)

            y_pred = model(test_batch)
            
            # ✅ Fixed prediction calculation for sigmoid output
            predictions = (torch.sigmoid(y_pred) > 0.5).float()
            
            correct += (predictions == test_labels).sum().item()
            total += test_labels.size(0)

    accuracy = 100 * correct / total
    print(f"🎯 Test Accuracy: {accuracy:.2f}%")
    return accuracy

# Run evaluation
print("📊 Evaluating model on training data...")
test_accuracy = evaluate_rnn_model()

📊 Evaluating model on training data...
🎯 Test Accuracy: 79.50%


In [12]:
import numpy as np

def softmax(arr):
    arr = np.array(arr)
    exp_arr = np.exp(arr - np.max(arr))  # for numerical stability
    return exp_arr * 100 / exp_arr.sum()

In [13]:
values = [10,20,30]
print(softmax(values))

[2.06106005e-07 4.53978686e-03 9.99954600e+01]


Using pre-trained bert model

In [214]:
from transformers import BertTokenizer, BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

c:\Users\91748\OneDrive\Desktop\pytorch\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [215]:
encodings = tokenizer(
    "hello world","my name is akash",
    max_length=8
)
encodings

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


{'input_ids': [101, 7592, 2088, 102, 2026, 2171, 2003, 102], 'token_type_ids': [0, 0, 0, 0, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [48]:
import torch
import warnings

warnings.filterwarnings("ignore")

In [49]:
ip = tokenizer(
    ["she is a gorgeous woman","how is this possible"],["i am him","this is impossible"],
    padding=True,
    truncation=True,
    max_length=128,
    return_tensors='pt'
)
for k,v in ip.items():
    print(k,'======>',v)

input_ids ======> tensor([[ 101, 2016, 2003, 1037, 9882, 2450,  102, 1045, 2572, 2032,  102],
        [ 101, 2129, 2003, 2023, 2825,  102, 2023, 2003, 5263,  102,    0]])
token_type_ids ======> tensor([[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0]])
attention_mask ======> tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]])


In [50]:
item = {key: torch.tensor(val[1]) for key, val in ip.items()}
for k,v in item.items():
    print(k,'======>',v)

input_ids ======> tensor([ 101, 2129, 2003, 2023, 2825,  102, 2023, 2003, 5263,  102,    0])
token_type_ids ======> tensor([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0])
attention_mask ======> tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0])


In [51]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Extract questions from train/test splits
train_q1 = [pair[0] for pair in x_train]
train_q2 = [pair[1] for pair in x_train] 
test_q1 = [pair[0] for pair in x_test]
test_q2 = [pair[1] for pair in x_test]

train_encodings = tokenizer(train_q1, train_q2, padding=True, truncation=True, max_length=128, return_tensors='pt')
train_dataset = CustomDataset(train_encodings, y_train)

test_encodings = tokenizer(test_q1, test_q2, padding=True, truncation=True, max_length=128, return_tensors='pt')
test_dataset = CustomDataset(test_encodings, y_test)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [52]:
len(test_dataset)

8000

In [55]:
from transformers.trainer import Trainer
from transformers.training_args import TrainingArguments
from sklearn.metrics import accuracy_score,f1_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred.predictions, eval_pred.label_ids
    # For binary classification with 2 labels
    preds = logits.argmax(-1)  # Get predicted class (0 or 1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds)
    return {"accuracy": acc, 'f1_score':f1}

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,          # 2-4 is guud
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=1e-5,          # Lower LR
    weight_decay=0.01,
    eval_strategy='steps',
    eval_steps=200,               # More frequent evaluation
    save_strategy="steps",
    save_steps=200,
    save_total_limit=1,
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # Your processed train CustomDataset
    eval_dataset=test_dataset,    # Your processed test CustomDataset
    compute_metrics=compute_metrics
)

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

model = BertForSequenceClassification.from_pretrained('./results/checkpoint-2000')
model = model.to(device)  # 🎯 Move model to GPU

# Update trainer with device-corrected model
trainer.model = model

Using device: cuda


In [56]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1 Score
200,No log,0.433818,0.776250,0.750696
400,No log,0.398466,0.801250,0.770893
600,0.453200,0.421198,0.790250,0.771015
800,0.453200,0.363325,0.830000,0.784127
1000,0.391600,0.398463,0.806875,0.783522
1200,0.391600,0.358857,0.840375,0.802597
1400,0.391600,0.363145,0.843250,0.802706
1600,0.311100,0.359652,0.843500,0.804436
1800,0.311100,0.361876,0.840875,0.806447
2000,0.308300,0.355948,0.845250,0.806017


TrainOutput(global_step=2000, training_loss=0.3660341110229492, metrics={'train_runtime': 16510.0123, 'train_samples_per_second': 3.876, 'train_steps_per_second': 0.121, 'total_flos': 4209776885760000.0, 'train_loss': 0.3660341110229492, 'epoch': 2.0})

In [ ]:
trainer.train(resume_from_checkpoint='./results/checkpoint-2000')

In [57]:
def evaluate_bert_model(trainer):
    results = trainer.evaluate()
    print(f"Test Loss: {results['eval_loss']:.4f}")
    print(f"Test Accuracy: {results['eval_accuracy']*100:.2f}%" if 'eval_accuracy' in results else "Accuracy not computed")
    print(f"F1 score: {results['eval_f1_score']*100:.2f}%" if 'eval_f1_score' in results else "F1 score not computed")
    return results

# Usage:
evaluate_bert_model(trainer)

Test Loss: 0.3559
Test Accuracy: 84.52%


{'eval_loss': 0.35594770312309265,
 'eval_accuracy': 0.84525,
 'eval_f1_score': 0.8060169225947978,
 'eval_runtime': 161.4725,
 'eval_samples_per_second': 49.544,
 'eval_steps_per_second': 1.548,
 'epoch': 2.0}

In [58]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

model_path = './results/checkpoint-2000'

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained(model_path)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device) # type:ignore
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [59]:
import torch.nn.functional as F

def predict_duplicate(question1, question2):
    """
    Predict if two questions are duplicates using the trained BERT model
    
    Args:
        question1 (str): First question
        question2 (str): Second question  
        model_path (str): Path to saved model checkpoint
        
    Returns:
        dict: Prediction results with probability and classification
    """
    
    # Tokenize input
    inputs = tokenizer(
        question1, 
        question2,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors='pt'
    )
    
    # Move to device
    inputs = {key: val.to(device) for key, val in inputs.items()}
    
    # Make prediction
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        
        # Get probabilities
        probabilities = F.softmax(logits, dim=-1)
        
        # Probability of being duplicate (class 1)
        duplicate_prob = probabilities[0][1].item()
    
    return {
        'duplicate_probability': duplicate_prob
    }

In [61]:
# Example usage:
result = predict_duplicate(
    "what is the process of coming late?",
    "what is the process of coming on time?"
)

print(f"Duplicate Probability: {result['duplicate_probability']:.4f}")

Duplicate Probability: 0.8638


In [61]:
def test_duplicate_questions():
    """
    Comprehensive test suite for duplicate question detection
    """
    
    # Test cases with expected results
    test_cases = [
        # 🎯 OBVIOUS DUPLICATES (Should be HIGH duplicate probability)
        {
            "q1": "How do I learn Python programming?",
            "q2": "What's the best way to learn Python?",
            "expected": "DUPLICATE",
            "category": "Programming - Similar Intent"
        },
        {
            "q1": "What is machine learning?",
            "q2": "Can you explain what machine learning is?",
            "expected": "DUPLICATE", 
            "category": "Definition - Same Concept"
        },
        {
            "q1": "How to lose weight fast?",
            "q2": "What's the quickest way to lose weight?",
            "expected": "DUPLICATE",
            "category": "Health - Same Goal"
        },
        {
            "q1": "Best restaurants in New York",
            "q2": "Top places to eat in NYC",
            "expected": "DUPLICATE",
            "category": "Location - Synonymous"
        },
        
        # 🔄 PARAPHRASES (Should be MODERATE-HIGH duplicate probability)
        {
            "q1": "How can I improve my English speaking skills?",
            "q2": "What are ways to get better at speaking English?",
            "expected": "DUPLICATE",
            "category": "Language Learning - Paraphrase"
        },
        {
            "q1": "Why is my computer running slow?",
            "q2": "What causes a computer to be sluggish?",
            "expected": "DUPLICATE",
            "category": "Technical - Different Wording"
        },
        
        # ❌ CLEAR NON-DUPLICATES (Should be LOW duplicate probability)
        {
            "q1": "How to cook pasta?",
            "q2": "What is quantum physics?",
            "expected": "NOT_DUPLICATE",
            "category": "Completely Different Topics"
        },
        {
            "q1": "Best programming language for beginners",
            "q2": "How to train a dog?",
            "expected": "NOT_DUPLICATE",
            "category": "Unrelated Domains"
        },
        {
            "q1": "What is the weather today?",
            "q2": "How to make coffee?",
            "expected": "NOT_DUPLICATE",
            "category": "Different Daily Activities"
        },
        
        # 🤔 TRICKY CASES (Edge cases that might confuse the model)
        {
            "q1": "How to make money online?",
            "q2": "How to lose money online?",
            "expected": "NOT_DUPLICATE",
            "category": "Opposite Intent - Key Word Difference"
        },
        {
            "q1": "Best Python IDE for Mac",
            "q2": "Best Python IDE for Windows", 
            "expected": "NOT_DUPLICATE",
            "category": "Similar Topic - Different Platform"
        },
        {
            "q1": "How to learn data science?",
            "q2": "How to teach data science?",
            "expected": "NOT_DUPLICATE",
            "category": "Related but Different Actions"
        },
        
        # 📚 EDUCATIONAL CONTEXT
        {
            "q1": "What are the fundamentals of machine learning?",
            "q2": "What are the basics of ML?",
            "expected": "DUPLICATE",
            "category": "Education - Abbreviation"
        },
        
        # 💼 BUSINESS/CAREER
        {
            "q1": "How to start a business?",
            "q2": "Steps to launch a startup",
            "expected": "DUPLICATE",
            "category": "Business - Similar Concept"
        },
        
        # 🔬 IDENTICAL QUESTIONS
        {
            "q1": "What is artificial intelligence?",
            "q2": "What is artificial intelligence?",
            "expected": "DUPLICATE",
            "category": "Exact Match"
        }
    ]
    
    print("🧪 DUPLICATE QUESTION DETECTION TEST SUITE")
    print("=" * 80)
    
    correct_predictions = 0
    total_tests = len(test_cases)
    results = []
    
    for i, test in enumerate(test_cases, 1):
        print(f"\n📋 Test {i}/{total_tests}: {test['category']}")
        print("-" * 60)
        
        # Get prediction
        result = predict_duplicate(test["q1"], test["q2"])
        
        # Determine model prediction
        model_prediction = "DUPLICATE" if result['duplicate_probability'] > 0.5 else "NOT_DUPLICATE"
        
        # Check if correct
        is_correct = model_prediction == test["expected"]
        correct_predictions += is_correct
        
        # Store results
        results.append({
            **test,
            **result,
            'model_prediction': model_prediction,
            'is_correct': is_correct
        })
        
        # Display results
        print(f"Q1: {test['q1']}")
        print(f"Q2: {test['q2']}")
        print(f"Expected: {test['expected']}")
        print(f"Model: {model_prediction}")
        print(f"Probability: {result['duplicate_probability']:.4f}")
        print(f"✅ Correct" if is_correct else "❌ Wrong")
    
    # Summary
    accuracy = correct_predictions / total_tests
    print("\n" + "=" * 80)
    print("📊 TEST SUMMARY")
    print("=" * 80)
    print(f"Total Tests: {total_tests}")
    print(f"Correct: {correct_predictions}")
    print(f"Wrong: {total_tests - correct_predictions}")
    print(f"Accuracy: {accuracy:.2%}")
    
    # Category-wise analysis
    print("\n📈 CATEGORY BREAKDOWN:")
    categories = {}
    for result in results:
        cat = result['category']
        if cat not in categories:
            categories[cat] = {'correct': 0, 'total': 0}
        categories[cat]['total'] += 1
        if result['is_correct']:
            categories[cat]['correct'] += 1
    
    for cat, stats in categories.items():
        acc = stats['correct'] / stats['total']
        print(f"  {cat}: {stats['correct']}/{stats['total']} ({acc:.1%})")
    
    return results

# Run the tests
test_results = test_duplicate_questions()

🧪 DUPLICATE QUESTION DETECTION TEST SUITE

📋 Test 1/15: Programming - Similar Intent
------------------------------------------------------------
Q1: How do I learn Python programming?
Q2: What's the best way to learn Python?
Expected: DUPLICATE
Model: DUPLICATE
Probability: 0.8759
✅ Correct

📋 Test 2/15: Definition - Same Concept
------------------------------------------------------------
Q1: What is machine learning?
Q2: Can you explain what machine learning is?
Expected: DUPLICATE
Model: DUPLICATE
Probability: 0.8121
✅ Correct

📋 Test 3/15: Health - Same Goal
------------------------------------------------------------
Q1: How to lose weight fast?
Q2: What's the quickest way to lose weight?
Expected: DUPLICATE
Model: DUPLICATE
Probability: 0.9128
✅ Correct

📋 Test 4/15: Location - Synonymous
------------------------------------------------------------
Q1: Best restaurants in New York
Q2: Top places to eat in NYC
Expected: DUPLICATE
Model: DUPLICATE
Probability: 0.5402
✅ Correct

📋 